In [9]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

async def main():
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://studerendeonline.dk/legate",
        )
        print(result.markdown)

await main()

[INIT].... → Crawl4AI 0.4.248
[FETCH]... ↓ https://studerendeonline.dk/legate... | Status: True | Time: 0.01s
[COMPLETE] ● https://studerendeonline.dk/legate... | Status: True | Total: 0.02s
×
####  Log ind
![](https://studerendeonline.dk/images/opengraph/jobbank-15.jpg)
×
#### Opret karriereprofil
Fortæl os lidt om dig selv og hjælp os med at matche dig med relevante job og karrieretilbud.
  * 1Uddannelse 
  * 2Dig selv 
  * 3Præferencer 


Hvad er din _højeste_ uddannelse? Er du fortsat under uddannelse, angiv gerne din højeste forventede titel samt anslået dimissionsdato. 
Uddannelsestitel * Anden erhvervsuddannelseAnden kort videregående udd.Anden lang videregående udd.Anden mellemlang videregående udd.Kurser, efteruddannelse og andetUdvekslingsopholdAdministrationsbachelorAdministrationsøkonomAdvokatAgriculture and EnvironmentAkademiingeniørAkademimerkonomAkademiuddannelse i HRAkademiuddannelse i ledelseAkademiøkonomAnimationsinstruktørAntropologAP Automotive TechnologyAP Chemical

In [2]:
from crawl4ai import AsyncWebCrawler, CacheMode, CrawlerRunConfig, JsonCssExtractionStrategy
import json
import polars as pl
import asyncio
import nest_asyncio

nest_asyncio.apply()


# Define the async function to extract scholarships
async def extract_scholarships():
    # Define the extraction schema
    schema = {
        "name": "Scholarships Extractor",
        "baseSelector": "div.card",
        "fields": [
            {
                "name": "scholarship_url",
                "selector": "div.card-block a.btn.btn-primary.btn-sm.btn-slim.pull-right",
                "type": "attribute",
                "attribute": "href",
            },
        ],
    }

    extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)

    # Configure the crawler
    config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        extraction_strategy=extraction_strategy,
    )

    all_scholarships = []

    # Iterate over pages 1 to 5
    for page in range(1, 6):
        url = f"https://studerendeonline.dk/legater/find/?cat=4&page={page}"
        async with AsyncWebCrawler() as crawler:
            result = await crawler.arun(
                url=url,
                config=config
            )
            # Parse the extracted content for each page
            scholarships = json.loads(result.extracted_content)
            all_scholarships.extend(scholarships)
            print(f"Extracted URLs from page {page}:")
            for scholarship in scholarships:
                print(scholarship["scholarship_url"])
            print(f"\nSuccessfully extracted {len(scholarships)} scholarship URLs from page {page}.")

    # Save the results in a dataframe
    df_scholarships = pl.DataFrame(all_scholarships)
    print(df_scholarships)
    return df_scholarships

# Run the extraction and save the DataFrame
async def main():
    global df_scholarships
    try:
        df_scholarships = await extract_scholarships()
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the async function in Jupyter Notebook
if __name__ == "__main__":
    asyncio.run(main())


[INIT].... → Crawl4AI 0.4.3b3
[FETCH]... ↓ https://studerendeonline.dk/legater/find/?cat=4&pa... | Status: True | Time: 4.67s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/find/?cat=4&pa... | Time: 134ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/find/?cat=4&pa... | Time: 0.06779500000016014s
[COMPLETE] ● https://studerendeonline.dk/legater/find/?cat=4&pa... | Status: True | Total: 4.88s
Extracted URLs from page 1:
/legater/10490/generelle-formal-herunder-kulturelle-og-sociale-formal
/legater/10491/studieophold-for-studerende-pa-kunstneriske-uddannelser
/legater/5041/studieophold-i-udlandet-kandidatniveau
/legater/25276/studieophold-i-udlandet-sygeplejestuderende
/legater/10215/studie-og-praktikophold-i-udlandet-for-alle-studieretninger
/legater/3162/studierejser-for-kandidatstuderende
/legater/10320/det-maritime-mba-legat
/legater/3326/forskningsophold-i-udlandet
/legater/24522/cp-relateret-forskning-og-praktisk-implementering
/legater/22137/uddannelse

In [3]:
df_scholarships

scholarship_url
str
"""/legater/10490/generelle-forma…"
"""/legater/10491/studieophold-fo…"
"""/legater/5041/studieophold-i-u…"
"""/legater/25276/studieophold-i-…"
"""/legater/10215/studie-og-prakt…"
…
"""/legater/26559/danske-kvinder-…"
"""/legater/9431/unge-med-tilknyt…"
"""/legater/9926/studerende-ved-k…"


In [4]:
# Assume df_scholarships is your polars DataFrame
base_url = "https://studerendeonline.dk"

# Concatenate the base URL with the partial URLs
df_scholarships = df_scholarships.with_columns(
    (base_url + df_scholarships['scholarship_url']).alias('full_url')
)

In [23]:
df_scholarships['full_url'].head(5).to_list()

['https://studerendeonline.dk/legater/10490/generelle-formal-herunder-kulturelle-og-sociale-formal',
 'https://studerendeonline.dk/legater/10491/studieophold-for-studerende-pa-kunstneriske-uddannelser',
 'https://studerendeonline.dk/legater/5041/studieophold-i-udlandet-kandidatniveau',
 'https://studerendeonline.dk/legater/25276/studieophold-i-udlandet-sygeplejestuderende',
 'https://studerendeonline.dk/legater/10215/studie-og-praktikophold-i-udlandet-for-alle-studieretninger']

In [7]:
df_scholarships.select('full_url').write_parquet('scholarship_urls.parquet')

In [ ]:
from crawl4ai import AsyncWebCrawler, CacheMode, CrawlerRunConfig
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy
import json
import pandas as pd
import asyncio
import nest_asyncio

# Set up asyncio for Jupyter Notebook
nest_asyncio.apply()

async def extract_details(urls):
    schema = {
        "name": "LegatExtractor",
        "baseSelector": "body",
        "fields": [
            {
                "name": "Legat",
                "selector": "h1.page-title",
                "type": "text",
                "transform": "strip"
            },
            {
                # Teaser text specifically in div.teaser.read-more
                "name": "Beskrivelse",
                "selector": "div.teaser.read-more",
                "type": "text",
                "transform": "strip"
            },
            {
                # Specifically targets the .textbox.box-blue that has a grantForwarder button
                # This ensures we get the correct box with the 'Ansøg via Legatbogen' button
                "name": "Frist",
                "selector": "div.textbox.box-blue.text-small:has(button.grantForwarder) b.text-bold",
                "type": "text",
                "transform": "strip"
            },
            {
                # Everything else (“Vi støtter” details, h2, etc.) 
                # is inside .col-sm-7.col-md-8
                # If you only want the row portion, use '> .row' 
                # or a more precise child selector
                "name": "Detaljer",
                "selector": "div.col-sm-7.col-md-8 > .row",
                "type": "text",
                "transform": "strip"
            }
        ]
    }
    
    extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)

    config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        extraction_strategy=extraction_strategy,
    )

    all_results = []

    async with AsyncWebCrawler() as crawler:
        for url in urls:
            result = await crawler.arun(url=url, config=config)
            if result.extracted_content is None:
                print(f"No content extracted from {url}")
                continue
            try:
                data = json.loads(result.extracted_content)
                all_results.append(data)
            except json.JSONDecodeError:
                print(f"Failed to decode JSON from {url}")

    return all_results

async def process_urls():
    # Adjust the DataFrame, column name, and slice as needed
    sample_urls = df_scholarships['full_url'].head(5).to_list()
    
    details_list = await extract_details(sample_urls)
    details_df = pd.DataFrame(details_list)
    return details_df

result_df = asyncio.run(process_urls())
print(result_df)


[INIT].... → Crawl4AI 0.4.248
[FETCH]... ↓ https://studerendeonline.dk/legater/10490/generell... | Status: True | Time: 6.63s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/10490/generell... | Time: 615ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/10490/generell... | Time: 2.6132866669940995s
[COMPLETE] ● https://studerendeonline.dk/legater/10490/generell... | Status: True | Total: 10.07s
[FETCH]... ↓ https://studerendeonline.dk/legater/10491/studieop... | Status: True | Time: 8.79s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/10491/studieop... | Time: 2323ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/10491/studieop... | Time: 5.948104959010379s
[COMPLETE] ● https://studerendeonline.dk/legater/10491/studieop... | Status: True | Total: 17.24s
[FETCH]... ↓ https://studerendeonline.dk/legater/5041/studieoph... | Status: True | Time: 5.78s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/5041/studieoph... | Time: 7

In [26]:
from crawl4ai import AsyncWebCrawler, CacheMode, CrawlerRunConfig
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy
import json
import asyncio
import nest_asyncio

# Set up asyncio for Jupyter Notebook
nest_asyncio.apply()

async def extract_details(urls):
    schema = {
        "name": "LegatExtractor",
        "baseSelector": "body",
        "fields": [
            {
                "name": "Legat",
                "selector": "h1.page-title",
                "type": "text",
                "transform": "strip"
            },
            {
                # Teaser text specifically in div.teaser.read-more
                "name": "Beskrivelse",
                "selector": "div.teaser.read-more",
                "type": "text",
                "transform": "strip"
            },
            {
                # Specifically targets the .textbox.box-blue that has a grantForwarder button
                # This ensures we get the correct box with the 'Ansøg via Legatbogen' button
                "name": "Frist",
                "selector": "div.textbox.box-blue.text-small:has(button.grantForwarder) b.text-bold",
                "type": "text",
                "transform": "strip"
            },
            {
                # Everything else (“Vi støtter” details, h2, etc.) 
                # is inside .col-sm-7.col-md-8
                # If you only want the row portion, use '> .row' 
                # or a more precise child selector
                "name": "Detaljer",
                "selector": "div.col-sm-7.col-md-8 > .row",
                "type": "text",
                "transform": "strip"
            }
        ]
    }
    
    extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)

    config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        extraction_strategy=extraction_strategy,
    )

    all_results = []

    async with AsyncWebCrawler() as crawler:
        for url in urls:
            result = await crawler.arun(url=url, config=config)
            if result.extracted_content is None:
                print(f"No content extracted from {url}")
                continue
            try:
                data = json.loads(result.extracted_content)
                all_results.append(data)
            except json.JSONDecodeError:
                print(f"Failed to decode JSON from {url}")

    return all_results

async def process_urls():
    # Adjust the DataFrame, column name, and slice as needed.
    # Here, assuming df_scholarships is defined elsewhere.
    sample_urls = df_scholarships['full_url'].head(5).to_list()
    
    details_list = await extract_details(sample_urls)
    return details_list

# Run the asynchronous processing and get the JSON results
results = asyncio.run(process_urls())

# Save the JSON data to a file
with open('results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

# Optionally, print the JSON results
print(json.dumps(results, ensure_ascii=False, indent=2))


[INIT].... → Crawl4AI 0.4.248
[FETCH]... ↓ https://studerendeonline.dk/legater/10490/generell... | Status: True | Time: 8.57s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/10490/generell... | Time: 1796ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/10490/generell... | Time: 4.015203792005195s
[COMPLETE] ● https://studerendeonline.dk/legater/10490/generell... | Status: True | Total: 15.95s
[FETCH]... ↓ https://studerendeonline.dk/legater/10491/studieop... | Status: True | Time: 7.41s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/10491/studieop... | Time: 4292ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/10491/studieop... | Time: 4.578331290991628s
[COMPLETE] ● https://studerendeonline.dk/legater/10491/studieop... | Status: True | Total: 18.11s
[FETCH]... ↓ https://studerendeonline.dk/legater/5041/studieoph... | Status: True | Time: 5.70s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/5041/studieoph... | Time: 2

In [8]:
from crawl4ai import AsyncWebCrawler, CacheMode, CrawlerRunConfig
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy
import json
import polars as pl
import asyncio
import nest_asyncio

# Set up asyncio for Jupyter Notebook
nest_asyncio.apply()

async def extract_details(urls):
    schema = {
        "name": "LegatExtractor",
        "baseSelector": "body",
        "fields": [
            {
                "name": "Legat",
                "selector": "h1.page-title",
                "type": "text",
                "transform": "strip"
            },
            {
                # Teaser text specifically in div.teaser.read-more
                "name": "Beskrivelse",
                "selector": "div.teaser.read-more",
                "type": "text",
                "transform": "strip"
            },
            {
                # Specifically targets the .textbox.box-blue that has a grantForwarder button
                # This ensures we get the correct box with the 'Ansøg via Legatbogen' button
                "name": "Frist",
                "selector": "div.textbox.box-blue.text-small:has(button.grantForwarder) b.text-bold",
                "type": "text",
                "transform": "strip"
            },
            {
                # Everything else (“Vi støtter” details, h2, etc.) 
                # is inside .col-sm-7.col-md-8
                # If you only want the row portion, use '> .row' 
                # or a more precise child selector
                "name": "Detaljer",
                "selector": "div.col-sm-7.col-md-8 > .row",
                "type": "text",
                "transform": "strip"
            }
        ]
    }
    
    extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)

    config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        extraction_strategy=extraction_strategy,
    )

    all_results = []

    async with AsyncWebCrawler() as crawler:
        for url in urls:
            result = await crawler.arun(url=url, config=config)
            if result.extracted_content is None:
                print(f"No content extracted from {url}")
                continue
            try:
                data = json.loads(result.extracted_content)
                all_results.append(data)
            except json.JSONDecodeError:
                print(f"Failed to decode JSON from {url}")

    return all_results

async def process_urls():
    
    details_list = await extract_details(df_scholarships['full_url'].to_list())
    # Save the list as a JSON file
    with open('details_list.json', 'w', encoding='utf-8') as f:
        json.dump(details_list, f, ensure_ascii=False, indent=2)
        
    details_df = pl.DataFrame(details_list)
    return details_df

result_df = asyncio.run(process_urls())
print(result_df)


[INIT].... → Crawl4AI 0.4.3b3
[FETCH]... ↓ https://studerendeonline.dk/legater/10490/generell... | Status: True | Time: 5.92s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/10490/generell... | Time: 192ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/10490/generell... | Time: 0.14984625000033702s
[COMPLETE] ● https://studerendeonline.dk/legater/10490/generell... | Status: True | Total: 6.28s
[FETCH]... ↓ https://studerendeonline.dk/legater/10491/studieop... | Status: True | Time: 2.54s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/10491/studieop... | Time: 146ms
[EXTRACT]. ■ Completed for https://studerendeonline.dk/legater/10491/studieop... | Time: 0.09591975000012098s
[COMPLETE] ● https://studerendeonline.dk/legater/10491/studieop... | Status: True | Total: 2.80s
[FETCH]... ↓ https://studerendeonline.dk/legater/5041/studieoph... | Status: True | Time: 4.15s
[SCRAPE].. ◆ Processed https://studerendeonline.dk/legater/5041/studieoph... | Time: 1

In [23]:
import json

# Read the JSON file
with open('details_list.json', 'r', encoding='utf-8') as f:
	content = f.read()
	if content:
		details_list = json.loads(content)
	else:
		details_list = []

# Print the JSON
print(json.dumps(details_list, indent=4, ensure_ascii=False))


[
    [
        {
            "Legat": "Augustinus Fonden",
            "Beskrivelse": "Augustinus Fonden er en af Danmarks store kulturfonde, som primært støtter projekter inden for Kunst & Kultur. Fonden støtter også Viden & Uddannelse samt sociale indsatser i Danmark og internationalt. Fondens filantropiske virke er baseret på fondens fundats fra 1942.",
            "Frist": "Løbende",
            "Detaljer": "Vi støtterKulturelle formål, herunder: Museer, bevaring af bygningsværker, klassisk musik, kulturarv, litteratur, scenekunst, billedkunst og kunsthåndværkForskningsprojekter indenfor Kulturarv samt nyresygdommeSociale indsatser i Danmark for særligt udsatte voksne og internationale humanitære formål i form af akut nødhjælpVi støtter IKKEEtablering og vedligehold af legepladser, forsamlings- og kulturhuse, sportsklubber, foreningslokaler etc.Studierejser for grupperStudier i DanmarkDeltagelse i konferencerPersonlige legater til forbedring af egne vilkårOpførsel eller restaureri

In [24]:
details_list

[[{'Legat': 'Augustinus Fonden',
   'Beskrivelse': 'Augustinus Fonden er en af Danmarks store kulturfonde, som primært støtter projekter inden for Kunst & Kultur. Fonden støtter også Viden & Uddannelse samt sociale indsatser i Danmark og internationalt. Fondens filantropiske virke er baseret på fondens fundats fra 1942.',
   'Frist': 'Løbende',
   'Detaljer': 'Vi støtterKulturelle formål, herunder: Museer, bevaring af bygningsværker, klassisk musik, kulturarv, litteratur, scenekunst, billedkunst og kunsthåndværkForskningsprojekter indenfor Kulturarv samt nyresygdommeSociale indsatser i Danmark for særligt udsatte voksne og internationale humanitære formål i form af akut nødhjælpVi støtter IKKEEtablering og vedligehold af legepladser, forsamlings- og kulturhuse, sportsklubber, foreningslokaler etc.Studierejser for grupperStudier i DanmarkDeltagelse i konferencerPersonlige legater til forbedring af egne vilkårOpførsel eller restaurering af ejendom uden offentlig adgangForkyndelse eller m

# Qdrant vector database setup
Link ("https://qdrant.tech/documentation/beginner-tutorials/search-beginners/")

In [44]:
from dotenv import load_dotenv, find_dotenv
from qdrant_client import QdrantClient
import os

load_dotenv(find_dotenv())

Qdrant_api_key = os.getenv("Qdrant_api_key")
client = QdrantClient(
    url="https://2c49a62c-f2c9-4788-808d-63746748ccb5.eu-central-1-0.aws.cloud.qdrant.io:6333", 
    api_key=Qdrant_api_key,
)

print(client.get_collections())

collections=[]


In [26]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [45]:
#encoder = SentenceTransformer("all-MiniLM-L6-v2")
encoder = SentenceTransformer("intfloat/multilingual-e5-large-instruct")

In [47]:
client.create_collection(
    collection_name="scholarship-vdb",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

In [39]:
import json

# Create the merged structure
merged_data = [
    {**detail[0], 'url': url}
    for detail, url in zip(details_list, df_scholarships['full_url'])
]

# Save to JSON file
with open('scholarships_data.json', 'w', encoding='utf-8') as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=2)

In [ ]:
# Print the first result from the details_list
print(json.dumps(details_list[0], indent=4, ensure_ascii=False))

In [41]:
import json

# Read the existing JSON file
with open('scholarships_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Create new structure with combined text field
vectorization_data = []
for entry in data:
    combined_text = f"{entry['Legat']}\n\n{entry['Beskrivelse']}\n\nDetaljeret information:\n{entry['Detaljer']}"
    
    vectorization_entry = {
        "Legat": combined_text,
        "Frist": entry["Frist"],
        "url": entry["url"],
    }
    vectorization_data.append(vectorization_entry)

# Save the modified structure
with open('vectorization_data.json', 'w', encoding='utf-8') as f:
    json.dump(vectorization_data, f, ensure_ascii=False, indent=2)

In [48]:

with open("vectorization_data.json", "r") as f:
    data = json.load(f)

documents = []
for item in data:
    if isinstance(item, list):
        documents.extend(item)
    else:
        documents.append(item)

client.upload_points(
    collection_name="scholarship-vdb",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Legat"]).tolist(),
            payload={
                "Legat": doc["Legat"],
                "Frist": doc["Frist"],
                "url": doc["url"],
            }
        )
        for idx, doc in enumerate(documents)
    ],
)

In [50]:
hits = client.query_points(
    collection_name="scholarship-vdb",
    query=encoder.encode("Vejle").tolist(),
    limit=5,
).points

for hit in hits:
    print(hit.payload, "score:", hit.score)

{'Legat': 'Fonden Herlufsholm Skoles Legater\n\nLegatets formål er ukendt.\n\nDetaljeret information:\nVi støtter ', 'Frist': 'Løbende', 'url': 'https://studerendeonline.dk/legater/15331/videreuddannelse-mv'} score: 0.8246202
{'Legat': "Jørgen Buch's Fond\n\nlegater til studieophold i udlandet\n\nDetaljeret information:\nVi støtter studieophold i udlandet", 'Frist': '15.05.2025', 'url': 'https://studerendeonline.dk/legater/11960/stotte-til-studerendes-studiebesog-i-udlandet'} score: 0.82174265
{'Legat': 'C.L. Davids Legat til minde om P. M. Gertz\n\nTil unge juridiske kandidaters videreuddannelse enten i udlandet eller herhjemme.\n\nDetaljeret information:\nVi støtter Unge juridiske kandidaters videreuddannelse enten i udlandet eller herhjemme', 'Frist': 'Ukendt', 'url': 'https://studerendeonline.dk/legater/3770/juridiske-kandidaters-videreuddannelse'} score: 0.8207586
{'Legat': 'Thorkild og Thea Rosenvolds Legatfond\n\nLegatets formål er at støtte unge sønderjyders studieophold i udla

# Langchain (RAG)

In [54]:
from dotenv import load_dotenv, find_dotenv
import os
import langsmith

load_dotenv(find_dotenv())
os.environ["LANGSMITH_TRACING"] = "true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT="scholarship-tracing"

In [57]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

In [61]:
from langchain_huggingface import HuggingFaceEmbeddings

# I embedded the data using:
# encoder = SentenceTransformer("intfloat/multilingual-e5-large-instruct")
# But now we'lll use hugging face, and not the SenceTransformers library directly

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")

In [63]:
from dotenv import load_dotenv, find_dotenv
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
import os

load_dotenv(find_dotenv())

Qdrant_api_key = os.getenv("Qdrant_api_key")
client = QdrantClient(
    url="https://2c49a62c-f2c9-4788-808d-63746748ccb5.eu-central-1-0.aws.cloud.qdrant.io:6333", 
    api_key=Qdrant_api_key,
)

print(client.get_collections())

vector_store = QdrantVectorStore(
    client=client,
    collection_name="scholarship-vdb",
    embedding=embeddings,
)

collections=[CollectionDescription(name='scholarship-vdb')]


In [78]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import os

os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm = HuggingFaceEndpoint(
    repo_id="DDSC/roberta-base-danish",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)

In [72]:
from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [79]:

from langchain.chains import RetrievalQA

# 5. Build the RetrievalQA chain
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 6. Query the chain
query = "Hvilke legater kan søges når min mor er ansat som socialrådgiver?"
response = qa_chain.run(query)
print(response)

/Users/lassedrud-lund/Documents/BIA - Business Intelligence Assosiation/Legat4me/crawl4ai/.venv/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


HfHubHTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://router.huggingface.co/hf-inference/models/DDSC/roberta-base-danish

In [67]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode


# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

NameError: name 'MessagesState' is not defined